In [59]:
import pickle
from random import randrange
import itertools
import random
import pandas as pd
import numpy as np
import math
from gams import *

######define the number of candidate charging stations
cs = 4

######define the maximum charging power of a bus
p_bus_max = 350

a1 = pickle.load(open('Feasible_chromosome_1.pickle','rb'))
a2 = pickle.load(open('Feasible_chromosome_2.pickle','rb'))
a3 = pickle.load(open('Feasible_chromosome_3.pickle','rb'))
a4 = pickle.load(open('Feasible_chromosome_4.pickle','rb'))
a5 = pickle.load(open('Feasible_chromosome_5.pickle','rb'))
a6 = pickle.load(open('Feasible_chromosome_6.pickle','rb'))

def Merge(dict1, dict2, dict3, dict4, dict5, dict6):
    res = {**dict1, **dict2, **dict3, **dict4, **dict5, **dict6}
    return res

init_population = Merge(a1, a2, a3, a4, a5, a6)

trip_information = pd.read_csv("Trip_information_NumericalStudy.csv")
terminal_distance = pd.read_csv("Between_terminal_distance_NumericalStudy.csv")
##terminals
#1---University Medical Center
#2---Salt Lake Central Station
#3---This is the Place State Park
#4---North Temple Station

From_depot_to_trip_time = pd.read_csv("From_depot_to_trip_time")
From_trip_to_depot_time = pd.read_csv("From_trip_to_depot_time")
From_depot_to_trip_energy = pd.read_csv("From_depot_to_trip_energy")
From_trip_to_depot_energy = pd.read_csv("From_trip_to_depot_energy")

available_charging_time = pd.read_csv("Between_trip_time_NumericalStudy.csv")
energy_need = pd.read_csv("Between_trip_energy_NumericalStudy.csv")

following_set = pickle.load(open('Following_set_dict.pickle','rb'))

#####################################
#define a function to get a following trip of a service trip
#####################################
def return_following(i,following_set):
    following_set_i = following_set[i]
    if len(following_set_i) == 0:
        return -1
    else:
        following_set_i = following_set_i[0:2]
        return(random.choice(following_set_i))

#####################################
#define a function to generate a block given a first trip
#####################################
def block_generation(i,block,following_set):
    while True:
        m = return_following(i,following_set)
        if m != -1: 
            block.append(m)
            i = m
        else: 
            #print("block=")
            #print(block)
            return block

####################################
#define a function to get difference of two lists
####################################
def diff(list1, list2):
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1))))

####################################
#define a function to generate the service trip part of a chromosome
####################################
def trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip):
    while True:
        if len(rest_trips) != 0:
            fist_trip = min(rest_trips)                        #get the trip with the earlest starting time in rest of the trips
            new_block = []
            new_block.append(fist_trip)
            new_block = block_generation(fist_trip,new_block,following_set)  #generate a new block
            rest_trips = diff(rest_trips, new_block)           #get rest of the trips
            chromosome = chromosome + new_block                #add the newly generated block to the chromosome
            trip_chains.append(new_block)
            no_bus += 1
            no_trip.append(len(new_block))
            following_set = {key:[ele for ele in following_set[key] if ele not in chromosome] for key in following_set.keys()}
        else:
            #print("All trips have been assigned to blocks")
            #print("Generated trip part of chromosome =")
            #print(trip_chains)
            #print("Fleet size under this trip schedule =")
            #print(no_bus)
            #print("Number of trips for each bus =")
            #print(no_trip)
            
            return trip_chains, no_bus, no_trip

##########################################
#define a function to generate the charging station part of the chromosome
##########################################
def charpart_chromosome_generation():
    Charging_station_part_chromosome = []
    for i in range(cs):
        Charging_station_part_chromosome.append(random.randint(0, 1))
    return Charging_station_part_chromosome

#########################################
#define a function to randomly assign a battery capacity for each bus in the bus fleet
#########################################
def battery_cap_generation(fleet_size):
    bus_type = [1,2]                   #available bus type in the market
    battery_type = [90,380]            #battery capacity of each bus type
    battery_size = [0]*fleet_size      #define a list to store the battery size of each bus in the bus fleet
    battery_lower = [0]*fleet_size     #define the lower bound of battery energy in bus
    battery_upper = [0]*fleet_size     #define the upper bound of battery energy in bus
    for i in range(fleet_size):        #randomly assign a battery capacity for each bus in the bus fleet
        battery_size[i] = random.choice(battery_type)
        battery_lower[i] =  battery_size[i]*0.2
        battery_upper[i] = battery_size[i]*0.9
    return battery_size, battery_lower, battery_upper

########################################
#define the function to check feasibility of the generated chromosome
########################################
def check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, charpart_chromosome):
    i = 0
    while (i < fleet_size):
        #print("i=" +str(i))
        a = trips_per_bus[i]                        #get the number of trips for the bus i
        battery_l = battery_lower[i]                #get the battery capacity lower bound of the bus i
        battery_u = battery_upper[i]                #get the battery capacity upper bound of the bus i
        trip_chain = trip_chains[i]                 #get the trip chain of the bus i
        b = trip_chain[0]                           #get the first trip in the trip chain of the bus i
        remain_energy = battery_u - From_depot_to_trip_energy.iloc[b-1][1] - trip_information.iloc[b-1]["Energy_need"]   #get the remaining energy in bus i after finishing its first trip
        if remain_energy < battery_l:
            #print("The depot to first trip part of the trip chain "+str(i)+" is not feasible")
            i = 100
            return 0
            break
        else:
            if a == 1:
                remain_energy = remain_energy - From_trip_to_depot_energy.iloc[b-1][1]
                if remain_energy < battery_l:
                    #print("The only trip to depot part of the trip chain "+str(i)+" is not feasible")
                    i = 100
                    return 0
                    break
                #else:
                    #print("Trip chain "+str(i)+" is feasible")
            elif a > 1:
                j = 1
                while (j < a):
                    #print("j= "+str(j))
                    bb = trip_chain[j] 
                    #print("bb= " +str(bb))
                    cc = trip_chain[j-1] 
                    #print("cc= " +str(cc))
                    remain_energy = remain_energy - energy_need.iloc[cc-1][bb]
                    if remain_energy < battery_l:
                        #print("The between trips part "+str(cc)+" to "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                        i = 100
                        return 0
                        break
                    else:
                        if j < a-1:
                            stt = trip_information.iloc[bb-1]["Start_terminal"]     #get the start terminal of the next trip
                            remain_energy = remain_energy + charpart_chromosome[int(stt)-1]*p_bus_max*available_charging_time.iloc[cc-1][bb]
                            if remain_energy > battery_u:
                                remain_energy = battery_u
                            else: 
                                remain_energy = remain_energy
                            remain_energy = remain_energy - trip_information.iloc[bb-1]["Energy_need"]
                            if remain_energy < battery_l:
                                #print("The between trips part "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 100
                                return 0
                                break
                        else:
                            remain_energy = remain_energy - From_trip_to_depot_energy.iloc[bb-1][1]
                            if remain_energy < battery_l:
                                #print("The last trip to depot part of the trip chain "+str(i)+" is not feasible")
                                i = 100
                                return 0
                                break
                            #else:
                                #print("Trip chain "+str(i)+" is feasible")
                    j += 1
        i += 1
    return 1

##########################################
#choose the smaller one from two items
##########################################
def compare_two(num1, num2):
    if num1 >= num2:
        return num2
    else:
        return num1
    
    
##########################################
#function for implementing the single-point crossover
##########################################
def crossover(l1, l2, k):
    l_new = []
    for i in range(0, k):
        l_new.append(l1[i])
    for j in range(k, len(l2)):
        l_new.append(l2[j])
    return l_new
        

##########################################
#crossover between two chromosomes
##########################################
def cross_over(chrom1, chrom2):
    off_spring = {}
    chrom1_char = chrom1["char_chromosome"]
    chrom2_char = chrom2["char_chromosome"]
    chrom1_trip = chrom1["trip_chromosome"]
    chrom2_trip = chrom2["trip_chromosome"]
    chrom1_batt = chrom1["battery_cap"]
    chrom2_batt = chrom2["battery_cap"]
    chrom1_flze = chrom1["fleet_size"]
    chrom2_flze = chrom2["fleet_size"]
    smaller_fleet_size = compare_two(chrom1_flze, chrom2_flze)
    mm = randrange(1,3)
    #print(mm)
    nn = randrange(1,smaller_fleet_size-1)
    #print(nn)
    off_char = crossover(chrom1_char, chrom2_char, mm)
    off_trip_raw = crossover(chrom1_trip, chrom2_trip, nn)
    off_flze_raw = len(off_trip_raw)
    
    #delete duplicate elements
    off_trip_fp = off_trip_raw[0:nn] #keep the first half part of the off_trip unchanged
    off_trip_sp = off_trip_raw[nn:off_flze_raw] #get the second half part of the off_trip
    off_trip_fp_com = list(itertools.chain.from_iterable(off_trip_fp)) #combine the elements in the first part of off_trip
    off_trip_sp_new = [[ele for ele in sub if ele not in off_trip_fp_com] for sub in off_trip_sp] #delete the elements in the second half part of the off_trip which are duplicated with the elements in the first half part
    off_trip_new = off_trip_fp + off_trip_sp_new #get the new off spring
    off_trip_new = list(filter(None, off_trip_new)) #delete empty trip chains in the new off spring
    
    #add missing elements
    off_trip_new_com = list(itertools.chain.from_iterable(off_trip_new)) #combine the elements in the new off spring
    all_trips = [*range(1,117,1)] #all service trips
    missing_trips = [ele for ele in all_trips if ele not in off_trip_new_com] #get the missing trips
    missing_trips.sort() #sort the missing trips and see it as a new trip chain
    new_trip_chain = missing_trips
    
    #get the trip part of the off spring
    off_trip_new.append(new_trip_chain) #append the new trip chain to the new off spring
    off_trip = off_trip_new
    
    #get the fleet size of the off spring
    off_flze = len(off_trip)
    
    #get the battery size for each bus
    off_batt = battery_cap_generation(off_flze)
    battery_lower = off_batt[1]
    battery_upper = off_batt[2]
    
    #get the number of trips of each bus
    trips_per_bus = []
    for ele in off_trip:
        trips_per_bus.append(len(ele))
    
    #check whether the off spring is feasible
    check = check_feasibility(off_flze, trips_per_bus, battery_lower, battery_upper, off_trip, off_char)
    #print(check)
    
    #store the offspring if it is feasible otherwise generate a new offspring
    while True:
        if check != 1:
            # generate the trip part of a new chromosome
            rest_trips = [i for i in range(117)]
            rest_trips.pop(0)
            chromosome = []
            no_bus = 0
            no_trip = []
            trip_chains = []
            trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)
        
            off_trip=trippart_chromosome[0]
            off_flze=trippart_chromosome[1]
            trips_per_bus=trippart_chromosome[2]
        
            # generate the charging station part of a new chromosome
            off_char = charpart_chromosome_generation()
        
            ### generate the battery capacity for each bus
            off_batt = battery_cap_generation(off_flze)
            battery_size = off_batt[0]
            battery_lower = off_batt[1]
            battery_upper = off_batt[2]
        
            ### check feasibility of the generated new chromosome
            check = check_feasibility(off_flze, trips_per_bus, battery_lower, battery_upper, off_trip, off_char)
            #print(check)
            
        else:
            off_spring = {"char_chromosome": off_char, "fleet_size": off_flze, "trip_chromosome": off_trip, "battery_cap": off_batt}
            break
            
    return off_spring

########################################
#mutation
########################################
def mutation(chrom):
    new_chrom = {}
    chrom_char = chrom["char_chromosome"]
    chrom_trip = chrom["trip_chromosome"]
    chrom_batt = chrom["battery_cap"]
    chrom_flze = chrom["fleet_size"]
    
    trips_per_bus = []
    for ele in chrom_trip:
        trips_per_bus.append(len(ele))
    
    battery_lower = chrom_batt[1]
    battery_upper = chrom_batt[2]
    
    n = random. randint(1,2) 
    #print("n=")
    #print(n)
    
    if n == 1:
        jj = random.randrange(len(chrom_char)) 
        #print("jj=")
        #print(jj)
        chrom_char[jj] = int(not chrom_char[jj])
    elif n == 2:
        ii = random.sample(chrom_trip,2) #randomly choose two trip chains in chrom_trip
        #print("ii=")
        #print(ii)
        res_trip_chains = [ele for ele in chrom_trip if ele not in ii] #delete ii from chrom_trip
        #print("res_trip_chains=")
        #print(res_trip_chains)
        res_trip_com = list(itertools.chain.from_iterable(res_trip_chains))
        #print("res_trip_com=")
        #print(res_trip_com)
        ii_com = list(itertools.chain.from_iterable(ii)) #combine the elements in ii
        #print("ii_com")
        #print(ii_com)
        
        chromosome = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        
        following_set_m = {key:[ele for ele in following_set[key] if ele not in res_trip_com] for key in following_set.keys()}
        #print("following_set_m=")
        #print(following_set_m)
        
        new_trip_chains = trippart_chromosome_generation(ii_com, chromosome, trip_chains, following_set_m, no_bus, no_trip)
        new_trip_chains = new_trip_chains[0]
        chrom_trip = res_trip_chains + new_trip_chains
        #print("chrom_trip=")
        #print(chrom_trip)
        chrom_flze = len(chrom_trip)
        #print("chrom_flze=")
        #print(chrom_flze)
        chrom_batt = battery_cap_generation(chrom_flze)
        
        battery_lower = chrom_batt[1]
        battery_upper = chrom_batt[2]
        
        #get the number of trips of each bus
        trips_per_bus = []
        for ele in chrom_trip:
            trips_per_bus.append(len(ele))
    
    #check feasibility of the new chromosome
    check = check_feasibility(chrom_flze, trips_per_bus, battery_lower, battery_upper, chrom_trip, chrom_char)
    #print("check=")
    #print(check)
    
    #store the new chromosome if it is feasible otherwise generate a new chromosome
    while True:
        if check != 1:
            # generate the trip part of a new chromosome
            rest_trips = [i for i in range(117)]
            rest_trips.pop(0)
            chromosome = []
            no_bus = 0
            no_trip = []
            trip_chains = []
            trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)
        
            chrom_trip=trippart_chromosome[0]
            chrom_flze=trippart_chromosome[1]
            trips_per_bus=trippart_chromosome[2]
        
            # generate the charging station part of a new chromosome
            chrom_char = charpart_chromosome_generation()
        
            ### generate the battery capacity for each bus
            chrom_batt = battery_cap_generation(chrom_flze)
            battery_size = chrom_batt[0]
            battery_lower = chrom_batt[1]
            battery_upper = chrom_batt[2]
        
            ### check feasibility of the generated new chromosome
            check = check_feasibility(chrom_flze, trips_per_bus, battery_lower, battery_upper, chrom_trip, chrom_char)
            #print("check=")
            #print(check)
        
        else:
            new_chrom = {"char_chromosome": chrom_char, "fleet_size": chrom_flze, "trip_chromosome": chrom_trip, "battery_cap": chrom_batt}
            break
            
    return new_chrom     
    
######################################
#genetic algorithm
######################################

###########select
def select_mating_pool(init_population, num_parents):
    fitness_list = []
    fitness_return_list = []
    pop_size = len(init_population)
    for i in range(1,pop_size+1):
        chromosome = init_population[i]
        Charging_station = chromosome["char_chromosome"]
        trip_chains = chromosome["trip_chromosome"]
        chrom_batt = chromosome["battery_cap"]
        fleet_size = chromosome["fleet_size"]
        ########import the parameters of the chromosome to gams
        # store trip chains to a dictionary
        dict_trip_set = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in trip_set:
                dict_trip_set[str(i),str(j)] = 1
        #print(dict_trip_set)

        # store the first trip of each bus to a dictionary
        dict_first_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            dict_first_trip[str(i), str(trip_set[0])] = 1
        #print(dict_first_trip)

        # store the last trip of each bus to a dictionary
        dict_last_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            dict_last_trip[str(i), str(trip_set[-1])] = 1
        #print(dict_last_trip)

        # store the adjacent trips of each bus to a dictionary
        dict_adjacent_trips = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in range(1, len(trip_set)):
                a = trip_set[j-1]
                b = trip_set[j]
                dict_adjacent_trips[str(i),str(a),str(b)] = 1
        #print(dict_adjacent_trips)

        # store the available time periods for charging before each trip of each bus to a dictionary
        dict_available_charging_time = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in range(1, len(trip_set)):
                trip1 = trip_set[j-1]
                trip2 = trip_set[j]
                trip1_endtime = trip_information.iloc[trip1-1]["End_time"]
                trip2_starttime = trip_information.iloc[trip2-1]["Start_time"]
                trip1_endterminal = trip_information.iloc[trip1-1]["End_terminal"]
                trip2_startterminal = trip_information.iloc[trip2-1]["Start_terminal"]
                between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
                between_time_need = math.ceil(between_distance/20*60)
                for l in range(int(trip1_endtime+between_time_need), int(trip2_starttime)):
                    dict_available_charging_time[str(i),str(trip2),str(int(trip2_startterminal)),str(l)] = 1
        #print(dict_available_charging_time)

        # store the available time periods for charging at the depot for each bus
        dict_available_charging_time_d = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            last_trip = trip_set[-1]
            first_trip = trip_set[0]
            last_trip_end_terminal = trip_information.iloc[last_trip-1]["End_terminal"]
            last_trip_end_time = trip_information.iloc[last_trip-1]["End_time"]
            first_trip_start_terminal = trip_information.iloc[first_trip-1]["Start_terminal"]
            first_trip_start_time = trip_information.iloc[first_trip-1]["Start_time"]
            arrive_depot_time = last_trip_end_time + math.ceil(From_trip_to_depot_time.iloc[last_trip-1][1]*60)
            leave_depot_time = first_trip_start_time - math.ceil(From_depot_to_trip_time.iloc[first_trip-1][1]*60)
            for t in range(int(arrive_depot_time),1441):
                dict_available_charging_time_d[str(i),str(t)] =1
            for t in range(1,int(leave_depot_time)):
                dict_available_charging_time_d[str(i),str(t)] =1
        #print(dict_available_charging_time_d)

        # store the start terminal of trip i to a dictionary
        dict_start_terminal = {}
        for i in range(116):
            start_terminal = trip_information.iloc[i]["Start_terminal"]
            dict_start_terminal[str(i+1), str(int(start_terminal))] = 1
        #print(dict_start_terminal)

        # store the end terminal of trip i to a dictionary
        dict_end_terminal = {}
        for i in range(116):
            end_terminal = trip_information.iloc[i]["End_terminal"]
            dict_end_terminal[str(i+1), str(int(end_terminal))] = 1
        #print(dict_end_terminal)

        # store the energy consumption of serving a trip of each bus to a dictionary
        dict_energy_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in trip_set:
                trip_energy_need = trip_information.iloc[j-1]["Energy_need"]
                dict_energy_trip[str(i),str(j)] = trip_energy_need
        #print(dict_energy_trip)

        # store the energy consumption of serving two adjacent trips of each bus to a dictionary
        dict_energy_between_trips = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in range(1, len(trip_set)):
                trip1 = trip_set[j-1]
                trip2 = trip_set[j]
                trip1_endterminal = trip_information.iloc[trip1-1]["End_terminal"]
                trip2_startterminal = trip_information.iloc[trip2-1]["Start_terminal"]
                between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
                between_energy_need = between_distance*2.69
                dict_energy_between_trips[str(i),str(trip1),str(trip2)] = between_energy_need
        #print(dict_energy_between_trips)

        # store the energy consumption from the depot to the first trip of each bus to a dictionary
        dict_energy_depot_to_first_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            first_trip = trip_set[0]
            dict_energy_depot_to_first_trip[str(i),str(first_trip)] = From_depot_to_trip_energy.iloc[first_trip-1][1]
        #print(dict_energy_depot_to_first_trip)

        # store the energy consumption from the last trip to the depot of each bus to a dictionary
        dict_energy_last_trip_to_depot = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            last_trip = trip_set[-1]
        #    print(last_trip)
            dict_energy_last_trip_to_depot[str(i),str(last_trip)] = From_trip_to_depot_energy.iloc[last_trip-1][1]
        #print(dict_energy_last_trip_to_depot)

        # store the total operating time per day per bus
        dict_operating_time = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            first_trip = trip_set[0]
            last_trip = trip_set[-1]
            time1 = From_depot_to_trip_time.iloc[first_trip-1][1]
            first_trip_start_time = trip_information.iloc[first_trip-1]["Start_time"]
            last_trip_end_time = trip_information.iloc[last_trip-1]["End_time"]
            time2 = last_trip_end_time - first_trip_start_time
            time3 = From_trip_to_depot_time.iloc[last_trip-1][1]
            dict_operating_time[str(i)] = time1*60 + time2 + time3*60
        #print(dict_operating_time)  

        # store whether a terminal is equipped with a charging station
        dict_term_charging = {}
        for i in range(cs):
            dict_term_charging[str(i+1)] = Charging_station[i]
        #print(dict_term_charging)

        ws = GamsWorkspace(system_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/", working_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS")
        # create new GamsDatabase instance
        db = ws.add_database()

        fleet_size_set = []
        i=1
        while i <= fleet_size:
            fleet_size_set.append(str(i))
            i += 1

        # add 1-dimensional set 'm' with explanatory text 'bus index' to the GamsDatabase
        m = db.add_set("m", 1, "bus index")
        for i in fleet_size_set:
            m.add_record(i)

        # add parameter 'batt' with domain 'm' with explanatory text 'battery capacity of bus bbb' to the GamsDatabase
        batt = db.add_parameter_dc("batt", [m], "battery capacity of bus bbb")
        for i in fleet_size_set:
            batt.add_record(i).value = chrom_batt[0][int(i)-1]

        # add parameter to define the trips that each bus serves
        bust = GamsParameter(db, "bust", 2, "whether a trip is served by a bus")
        for k, v in dict_trip_set.items():
            bust.add_record(k).value = v

        # add parameter to define the first trip of each bus
        busf = GamsParameter(db, "busf", 2, "whether a trip is the first trip of a bus")
        for k, v in dict_first_trip.items():
            busf.add_record(k).value = v

        # add parameter to define the last trip of each bus
        busl = GamsParameter(db, "busl", 2, "whether a trip is the last trip of a bus")
        for k, v in dict_last_trip.items():
            busl.add_record(k).value = v

        # add parameter to define the two adjacent trips of each bus
        busaj = GamsParameter(db, "busaj", 3, "whether two trips are adjacent trips of a bus")
        for k, v in dict_adjacent_trips.items():
            busaj.add_record(k).value = v

        # add parameter to define the available charging time periods before serving a trip of each bus
        busct = GamsParameter(db, "busct", 4, "available charging time periods before serving a trip of a bus")
        for k, v in dict_available_charging_time.items():
            busct.add_record(k).value = v

        # add parameter to define the available charging time periods at the depot for each bus
        busctd = GamsParameter(db, "busctd", 2, "available charging time periods at the depot of a bus")
        for k, v in dict_available_charging_time_d.items():
            busctd.add_record(k).value = v

        # add parameter to define the start terminal of each trip
        tripst = GamsParameter(db, "tripst", 2, "start terminal of a trip")
        for k, v in dict_start_terminal.items():
            tripst.add_record(k).value = v

        # add parameter to define the end terminal of each trip
        tripet = GamsParameter(db, "tripet", 2, "end terminal of a trip")
        for k, v in dict_end_terminal.items():
            tripet.add_record(k).value = v

        # add parameter to define the energy need for serving a trip of each bus
        busen = GamsParameter(db, "busen", 2, "energy need for serving a trip of a bus")
        for k, v in dict_energy_trip.items():
            busen.add_record(k).value = v

        # add parameter to define the energy consumption between two adjacent trips of each bus
        busea = GamsParameter(db, "busea", 3, "energy consumption between two adjacent trips of a bus")
        for k, v in dict_energy_between_trips.items():
            busea.add_record(k).value = v

        # add parameter to define the energy consumption from the depot to the first trip of each bus
        busdf = GamsParameter(db, "busdf", 2, "energy consumption from the depot to the first trip of a bus")
        for k, v in dict_energy_depot_to_first_trip.items():
            busdf.add_record(k).value = v

        # add parameter to define the energy consumption from the last trip to the depot of each bus
        busld = GamsParameter(db, "busld", 2, "energy consumption from the last trip to the depot of a bus")
        for k, v in dict_energy_last_trip_to_depot.items():
            busld.add_record(k).value = v

        # add parameter to define the operating time of each bus
        busst = GamsParameter(db, "busst", 1, "operating time of each bus")
        for k, v in dict_operating_time.items():
            busst.add_record(k).value = v

        # add parameter to define whether a terminal is equipped with a charging station
        terch = GamsParameter(db, "terch", 1, "whether a terminal is equipped with a charging station")
        for k, v in dict_term_charging.items():
            terch.add_record(k).value = v

        # export the GamsDatabase to a GDX file with name 'data.gdx' located in the 'working_directory' of the GamsWorkspace
        db.export("/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/data.gdx")

        # call and run gams model to calculate the fitness value of the chromosome
        gams_model = ws.add_job_from_file("Ebus_scheduling_new.gms")
        gams_model.run()
        if (gams_model.out_db["modelStat"].find_record().value == 1 and gams_model.out_db["solveStat"].find_record().value == 1):
            for rec in gams_model.out_db["objvalue"]:
                fit_value = -rec.level
        else:
            fit_value = -999999999
        print(fit_value)
            
        fitness_list.append(fit_value)
        fitness_return_list.append(fit_value)

    fitness = np.array(fitness_list)
    fitness_return = np.array(fitness_return_list)
    #print(fitness_return)

    parents = {}
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num] = init_population[max_fitness_idx+1]
        fitness[max_fitness_idx] = -99999999999
    return parents,fitness_return 

###########crossover
def crossover_g(parents, offspring_size):
    num_parents = len(parents)
    offspring = {}
    for k in range(offspring_size):
        # Index of the first parent to mate.
        parent1_idx = k%num_parents
        # Index of the second parent to mate.
        parent2_idx = (k+1)%num_parents
        parent1 = parents[parent1_idx]
        parent2 = parents[parent2_idx]
        off_spring = cross_over(parent1, parent2)
        offspring[k] = off_spring
    return offspring

###########mutation
def mutation_g(offspring_crossover,mutationRate):
    num_mutation = int(mutationRate*len(offspring_crossover))
    for idx in range(num_mutation):
        mutation_idx = idx%num_mutation
        offspring_crossover[mutation_idx] = mutation(offspring_crossover[mutation_idx])
    return offspring_crossover

###########genetic algorithm
maximal_generation=150
population_size=120
parent_number=36
#maximal_generation=50
#population_size=20
#parent_number=16
mutation_rate=0.15
#number_of_variables = 3 #Battery size, charger number, charger power, charging threshold    
child_number = population_size - parent_number



In [60]:
new_population = init_population
best_outputs = []
for i in range(maximal_generation):
    print("Generation: "+str(i))
    parents,fitness_return = select_mating_pool(new_population, parent_number)
    offspring_crossover = crossover_g(parents, offspring_size=child_number)
    offspring_mutation = mutation_g(offspring_crossover, mutation_rate)
    for key,value in parents.items():
        new_population[key+1] = value
    for key,value in offspring_mutation.items():
        new_population[key+parent_number+1] = value

    print(fitness_return)
    print(np.max(fitness_return))


    max_fitness_idx = np.where(fitness_return == np.max(fitness_return))
    max_fitness_idx = max_fitness_idx[0][0]
    best_solution = (np.max(fitness_return),new_population[max_fitness_idx+1])
    best_outputs.append(best_solution)

#print(best_solution)
    

Generation: 0
-3924.947654125619
-4305.600287927911
-4148.068062862993
-4199.96122939513
-3915.961530255712
-4091.2941297280113
-999999999
-3892.425202947791
-3975.8903760409407
-3924.4709684143245
-4264.466189292537
-3943.687374127986
-3887.133867369185
-999999999
-4050.813764181273
-4214.29537370744
-3957.8752034898644
-3783.541431050832
-4251.969178590244
-999999999
-3949.4001703907907
-999999999
-4173.797601094316
-3847.758455480168
-4062.9520686050873
-999999999
-3801.381855396699
-3968.8490283106503
-3706.0064682365155
-4104.992126528287
-3981.7455750172066
-3957.3496006342657
-4102.844560514361
-3896.5185975195054
-4184.512168018946
-3989.352767143231
-4055.030661032102
-3930.7911238340307
-4047.024648762032
-4064.922638792552
-4419.989218399003
-4739.879437658994
-4629.41268277625
-4295.9847864388075
-4464.530416958236
-4508.131393154215
-4396.799854874671
-4549.1574914669845
-4562.7261134419505
-4287.288506882043
-4510.641336622145
-4376.155499493816
-4273.692797669772
-4151.6

-3706.0064682365155
-3783.541431050832
-3801.381855396699
-3809.903114308725
-3839.6382773327855
-3847.758455480168
-3864.881299009251
-3879.4902668991554
-3885.3665704804907
-3887.133867369185
-3890.590864292965
-3891.2886315253886
-3892.425202947791
-3896.5185975195054
-3902.657162453287
-3915.961530255712
-3916.4514059359317
-3917.997796209965
-3920.394601561651
-3921.2488307998556
-3924.4709684143245
-3924.947654125619
-3930.7911238340307
-3931.5937317639814
-3934.9071786096583
-3939.9870513621618
-3941.2681148310444
-3943.687374127986
-3944.639844964901
-3947.9901358989637
-3949.4001703907907
-3952.8602303067123
-3957.3496006342657
-3957.8752034898644
-3958.928512472814
-3961.3674297043735
-4127.113604402181
-4347.359598418503
-3651.5344815249673
-4127.482916350184
-3937.157883699975
-3909.5417322529274
-4297.137483495426
-4045.4739712282158
-3935.4019577244358
-4175.747120705278
-3949.348020213738
-4061.8044378604973
-3726.47921210282
-4197.787864721733
-3954.6786861514965
-4092.

-3651.5344815249673
-3706.0064682365155
-3719.6630383250235
-3726.47921210282
-3735.8126518312465
-3750.1825602605213
-3767.114953710029
-3769.428984131409
-3778.4713147057096
-3783.541431050832
-3801.381855396699
-3809.124038847319
-3809.903114308725
-3824.6132887786125
-3829.9483313620276
-3835.161555691408
-3839.6382773327855
-3843.8711906823655
-3847.758455480168
-3855.05916175403
-3856.2085689111514
-3858.717870352496
-3864.1187209323703
-3864.881299009251
-3868.8500889368124
-3879.4902668991554
-3880.152098859063
-3885.3665704804907
-3885.3678603246262
-3886.164237267958
-3887.133867369185
-3890.590864292965
-3890.8205422723304
-3891.2886315253886
-3892.425202947791
-3894.621415636163
-3892.6316188600313
-3936.253576361488
-4165.545972061888
-3955.7218201472097
-3961.0518895856235
-3826.627559272606
-3955.642953640881
-3810.7068299433204
-999999999
-4039.958153933342
-4263.938623752909
-3982.108208846692
-4173.632396025521
-3960.7983454839505
-999999999
-3916.5688592759825
-3975.

-3651.5344815249673
-3693.2397881962333
-3706.0064682365155
-3707.566703880925
-3714.015895551705
-3718.5957739738787
-3719.6630383250235
-3726.47921210282
-3735.8126518312465
-3750.1825602605213
-3767.114953710029
-3769.428984131409
-3778.4713147057096
-3783.541431050832
-3787.9627677388125
-3801.381855396699
-3806.797058934394
-3809.124038847319
-3809.903114308725
-3810.7068299433204
-3823.2578960230826
-3824.6132887786125
-3826.627559272606
-3829.9483313620276
-3835.161555691408
-3839.6382773327855
-3843.8711906823655
-3844.9489429224236
-3846.1138357091186
-3847.758455480168
-3853.7565722786267
-3855.05916175403
-3856.2085689111514
-3858.717870352496
-3861.484270822658
-3862.4613673147924
-3983.6693236979036
-3929.3153400258198
-4027.163281931461
-4323.36874576595
-3947.4006303312153
-4227.764584044728
-4011.8028128584137
-4241.754696694948
-3965.07323207995
-3888.7481237193797
-4044.8945772643997
-4204.945556537452
-4182.069556966107
-4010.0971304243626
-4037.481301060654
-4059.73

-3651.5344815249673
-3676.090334272639
-3693.2397881962333
-3706.0064682365155
-3707.566703880925
-3714.015895551705
-3718.5957739738787
-3719.6630383250235
-3726.47921210282
-3735.8126518312465
-3750.1825602605213
-3767.114953710029
-3769.2358988899728
-3769.428984131409
-3773.761047941627
-3778.4713147057096
-3780.520818412283
-3783.541431050832
-3787.9627677388125
-3801.381855396699
-3806.797058934394
-3809.124038847319
-3809.903114308725
-3810.7068299433204
-3823.2578960230826
-3824.6132887786125
-3825.6635566799805
-3826.627559272606
-3829.4348669927385
-3829.905237057645
-3829.9483313620276
-3833.910276768542
-3835.161555691408
-3839.6382773327855
-3843.8711906823655
-3844.157234767442
-3994.923291466173
-4002.0898252060374
-3997.831089447939
-3929.8193453607873
-4171.1577425371315
-4609.4260752698265
-3944.4007462437903
-3991.1711613442917
-4118.809417753542
-4111.565852306699
-4126.357266129645
-3874.80637308508
-3962.0220581375524
-4332.6801064155325
-999999999
-4262.584946242

...

-3593.365883076295
-3595.3501967515217
-3601.0014143640683
-3611.137252442108
-3614.5380909241294
-3635.0244130631054
-3636.6018668307424
-3639.107134455811
-3640.5692441833503
-3642.392757524929
-3643.2614781632547
-3645.567088939557
-3646.036326776119
-3651.5344815249673
-3652.3753225178893
-3654.251811298689
-3654.9869513542026
-3657.6801489788922
-3659.807921542416
-3664.4639212992797
-3666.9390673486887
-3667.4485907195685
-3670.5381939035524
-3672.977286748602
-3673.339315413827
-3673.946262523106
-3675.1790522615684
-3675.206901170658
-3676.081242633224
-3676.090334272639
-3676.2576671618845
-3676.9245161879717
-3677.054299826068
-3678.4809433171795
-3679.8152731031955
-3685.4000275679336
-3941.7377815812497
-4260.110139186092
-4006.1838636613816
-4011.901759015714
-4304.893529922416
-4073.7376008142082
-3893.9980797890025
-4010.4602728931523
-3891.8935557064237
-4222.010309562361
-3958.941980539883
-4220.460743604312
-3873.6811465603205
-4134.004148442313
-4011.564123286178
-99

-3593.365883076295
-3595.3501967515217
-3601.0014143640683
-3611.137252442108
-3614.5380909241294
-3635.0244130631054
-3636.6018668307424
-3639.107134455811
-3640.5692441833503
-3642.392757524929
-3643.2614781632547
-3645.567088939557
-3645.759036710955
-3646.036326776119
-3651.5344815249673
-3652.3753225178893
-3654.251811298689
-3654.9869513542026
-3657.6801489788922
-3659.807921542416
-3664.4639212992797
-3666.9390673486887
-3667.4485907195685
-3670.5381939035524
-3672.977286748602
-3673.339315413827
-3673.946262523106
-3675.1790522615684
-3675.206901170658
-3676.081242633224
-3676.090334272639
-3676.2576671618845
-3676.9245161879717
-3677.054299826068
-3678.4809433171795
-3679.8152731031955
-4115.478612132597
-4203.821421297052
-4218.017001367189
-3886.366304702838
-4118.341173902484
-4413.588873423174
-4016.0663582846514
-999999999
-4217.662577823528
-4424.776903154849
-4108.175433736723
-3864.2023305426446
-4110.772545451763
-4035.3265275520052
-3958.3291167393427
-4323.654640093

-3593.365883076295
-3595.3501967515217
-3601.0014143640683
-3611.137252442108
-3614.5380909241294
-3635.0244130631054
-3636.6018668307424
-3639.107134455811
-3640.5692441833503
-3642.392757524929
-3643.2614781632547
-3645.567088939557
-3645.759036710955
-3646.036326776119
-3651.5344815249673
-3652.3753225178893
-3654.251811298689
-3654.9869513542026
-3657.6801489788922
-3659.807921542416
-3664.4639212992797
-3666.9390673486887
-3667.4485907195685
-3670.5381939035524
-3672.977286748602
-3673.339315413827
-3673.946262523106
-3675.1790522615684
-3675.206901170658
-3676.081242633224
-3676.090334272639
-3676.2576671618845
-3676.9245161879717
-3677.054299826068
-3678.4809433171795
-3679.8152731031955
-4385.536880822144
-4470.511599582517
-3947.9891513813745
-4293.061837046956
-4343.698226222888
-4061.6233109852496
-3908.603308400534
-4298.533881636825
-3919.117447894904
-999999999
-4045.4240935004314
-3846.2271829307747
-4096.8391070580965
-4148.2838787625415
-4203.8639748489095
-4020.481489

-3593.365883076295
-3595.3501967515217
-3601.0014143640683
-3611.137252442108
-3614.5380909241294
-3635.0244130631054
-3636.6018668307424
-3639.107134455811
-3640.5692441833503
-3642.392757524929
-3643.2614781632547
-3645.567088939557
-3645.759036710955
-3646.036326776119
-3651.5344815249673
-3652.3753225178893
-3654.251811298689
-3654.9869513542026
-3657.6801489788922
-3659.807921542416
-3664.4639212992797
-3666.9390673486887
-3667.4485907195685
-3669.614524499847
-3670.5381939035524
-3672.977286748602
-3673.339315413827
-3673.946262523106
-3675.1790522615684
-3675.206901170658
-3676.081242633224
-3676.090334272639
-3676.2576671618845
-3676.9245161879717
-3677.054299826068
-3678.4809433171795
-3916.6195176946685
-3777.6354759357746
-4102.751487322949
-3899.28321976404
-3838.665397888497
-4057.2706264416315
-4054.6508083574904
-4000.607666411963
-3689.0211587354092
-3822.772383464115
-4322.7907936135725
-999999999
-3852.3648592808113
-999999999
-4155.167571474616
-4285.125376964004
-40

-3593.365883076295
-3595.3501967515217
-3601.0014143640683
-3611.137252442108
-3614.5380909241294
-3635.0244130631054
-3636.6018668307424
-3639.107134455811
-3640.5692441833503
-3642.392757524929
-3643.2614781632547
-3645.567088939557
-3645.759036710955
-3646.036326776119
-3651.5344815249673
-3652.3753225178893
-3654.251811298689
-3654.9869513542026
-3657.6801489788922
-3659.807921542416
-3664.4639212992797
-3666.9390673486887
-3667.4485907195685
-3669.614524499847
-3670.5381939035524
-3672.977286748602
-3673.339315413827
-3673.946262523106
-3675.1790522615684
-3675.206901170658
-3676.081242633224
-3676.090334272639
-3676.2576671618845
-3676.9245161879717
-3677.054299826068
-3678.4809433171795
-4007.9148786130495
-4432.485851752929
-3807.0541418247476
-4121.796138516571
-3994.2222732047603
-4250.135758717472
-3875.214235799289
-3829.109812068573
-3720.85237805952
-3857.559853762761
-4257.307953778381
-4110.790803898015
-3945.5059263781886
-3943.531894111942
-3889.1544101331583
-3696.89

In [61]:
print(best_solution)

(-3593.365883076295, {'char_chromosome': [1, 1, 1, 1], 'fleet_size': 8, 'trip_chromosome': [[1, 4, 9, 16, 23, 28, 33, 38, 43, 50, 59, 64, 71, 77, 81, 86, 91, 96, 104, 107, 110, 114, 116], [2, 8, 14, 19, 24, 31, 36, 41, 48, 56, 63, 70, 75, 82, 92, 97, 103, 109, 113], [3, 5, 11, 18, 27, 34, 42, 52, 57, 65, 72, 80, 88, 95, 101, 108, 112, 115], [6, 7, 17, 25, 32, 39, 46, 51, 58, 67, 76, 83, 90, 99, 105, 111], [10, 21, 29, 37, 44, 49, 60, 68, 74, 85, 89, 94, 98, 106], [12, 20, 30, 35, 40, 47, 54, 62, 66, 79, 87, 93, 100], [13, 26, 45, 53, 61, 73, 84, 102], [15, 22, 55, 69, 78]], 'battery_cap': ([380, 380, 380, 380, 380, 380, 90, 90], [76.0, 76.0, 76.0, 76.0, 76.0, 76.0, 18.0, 18.0], [342.0, 342.0, 342.0, 342.0, 342.0, 342.0, 81.0, 81.0])})


In [48]:
new_population = a2
parent_number = 2
parents,fitness_return = select_mating_pool(new_population, parent_number)

KeyError: 1

In [57]:
a3

{41: {'char_chromosome': [1, 1, 1, 1],
  'fleet_size': 10,
  'trip_chromosome': [[1,
    4,
    10,
    20,
    27,
    32,
    39,
    44,
    51,
    56,
    63,
    71,
    78,
    82,
    90,
    100,
    107,
    113,
    116],
   [2, 8, 15, 24, 33, 41, 49, 54, 59, 66, 75, 83, 88, 95, 101, 108, 112, 115],
   [3, 9, 14, 22, 35, 40, 50, 61, 68, 74, 85, 92, 97, 105, 111, 114],
   [5, 12, 16, 26, 36, 45, 55, 60, 69, 77, 86, 91, 96, 104, 110],
   [6, 13, 18, 29, 37, 43, 48, 58, 67, 76, 87, 94, 103, 109],
   [7, 19, 30, 38, 47, 57, 62, 72, 81, 89, 98, 106],
   [11, 17, 28, 42, 53, 65, 73, 84, 99],
   [21, 25, 46, 70, 93, 102],
   [23, 34, 79],
   [31, 52, 64, 80]],
  'battery_cap': ([90, 380, 380, 380, 380, 380, 90, 380, 380, 90],
   [18.0, 76.0, 76.0, 76.0, 76.0, 76.0, 18.0, 76.0, 76.0, 18.0],
   [81.0, 342.0, 342.0, 342.0, 342.0, 342.0, 81.0, 342.0, 342.0, 81.0])},
 42: {'char_chromosome': [1, 1, 1, 0],
  'fleet_size': 11,
  'trip_chromosome': [[1,
    5,
    10,
    18,
    26,
    3